In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.125      0.125      0.125
  0.         0.125      0.         0.         0.         0.
  0.         0.         0.         0.125      0.1        0.1
  0.         0.        ]
 [0.         0.         0.11111111 0.11111111 0.11111111 0.
  0.11111111 0.         0.1        0.         0.         0.
  0.         0.         0.         0.11111111 0.         0.1
  0.         0.11111111]
 [0.         0.11111111 0.         0.         0.14285714 0.14285714
  0.         0.         0.         0.         0.14285714 0.
  0.         0.         0.         0.125      0.         0.
  0.         0.14285714]
 [0.125      0.11111111 0.         0.         0.         0.
  0.125      0.         0.1        0.         0.         0.
  0.1        0.         0.         0.         0.         0.1
  0.         0.125     ]
 [0.125      0.11111111 0.14285714 0.         0.         0.
  0.         0.         0.1        0.         0.         0.
  0.         0.         0.         0.         

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[5, 7, 8, 6, 7, 4, 3, 3, 3, 9, 2, 5, 4, 9, 4, 4, 7, 9, 9, 2]
2
3
4
9
19


In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        #new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        #vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])


16 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(7.888

28 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
30 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
30 tensor(7.62

torch.Size([50, 1])
torch.Size([50, 1])
44 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
44 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
45 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
45 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
45 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
45 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50,

59 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
59 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
61 tensor(12.57

74 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
74 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
74 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
75 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
75 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
75 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
75 tensor(7.888

89 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
89 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
89 tensor(7.8884, grad_fn=<MseLossBackward0>) tensor([ 3.2934, -3.8799]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(12.5751, grad_fn=<MseLossBackward0>) tensor([ 6.8512, -4.4819]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(7.6240, grad_fn=<MseLossBackward0>) tensor([ 3.6137, -3.2920]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(9.7475, grad_fn=<MseLossBackward0>) tensor([ 5.6347, -3.1293]) tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(7.888

In [11]:
parameters_to_vector(model.parameters())

tensor([ 0.2730, -0.1370], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3
                                   )

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, tensor([[1.7150, 0.0000],
        [0.0000, 1.7150]]), tensor([[-0.0422,  0.0000],
        [ 0.0000, -0.0422]]), tensor([[2.1969, 0.0000],
        [0.0000, 2.1969]]), 0, 0, 0, 0, tensor([[1.7599, 0.0000],
        [0.0000, 1.7599]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, tensor([[1.1608, 0.0000],
        [0.0000, 1.1608]])]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.0935,  0.0979])


tensor([-0.1870,  0.1958], grad_fn=<CatBackward0>)

In [19]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:00<04:26,  7.51it/s]

Training_loss 9.40203,   Relative Error 1.11623


  0%|          | 2/2000 [00:00<04:15,  7.81it/s]

Training_loss 9.38445,   Relative Error 1.11508


  0%|          | 3/2000 [00:00<05:53,  5.65it/s]

...

Training_loss 0.04991,   Relative Error 0.07769
Training_loss 0.04990,   Relative Error 0.07768


100%|█████████▉| 1998/2000 [03:26<00:00,  7.61it/s]

Training_loss 0.04989,   Relative Error 0.07767
Training_loss 0.04976,   Relative Error 0.07757


100%|██████████| 2000/2000 [03:26<00:00,  9.67it/s]

Training_loss 0.04969,   Relative Error 0.07751
Training_loss 0.04966,   Relative Error 0.07749


In [20]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.7920,  1.9307], grad_fn=<CatBackward0>)

In [21]:
for j in G.neighbors(0):
    print(j)

2
3
4
9
19


In [22]:
parameters_to_vector(models[0].parameters())

tensor([1.7862, 1.9280], grad_fn=<CatBackward0>)

In [23]:
projection_list[0]

[0,
 0,
 tensor([[1.7150, 0.0000],
         [0.0000, 1.7150]]),
 tensor([[-0.0422,  0.0000],
         [ 0.0000, -0.0422]]),
 tensor([[2.1969, 0.0000],
         [0.0000, 2.1969]]),
 0,
 0,
 0,
 0,
 tensor([[1.7599, 0.0000],
         [0.0000, 1.7599]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([[1.1608, 0.0000],
         [0.0000, 1.1608]])]

In [24]:
projected_weights[0]

[0,
 0,
 tensor([-0.8066, -0.6668]),
 tensor([-0.0103, -0.0174]),
 tensor([-0.9049, -0.2024]),
 0,
 0,
 0,
 0,
 tensor([-0.0352,  0.1558]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([0.6834, 0.2581])]

In [25]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [26]:
np.save( 'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), total_rel_error)

In [27]:
'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'training_loss_dfedu0_01_pout0_2'

In [28]:
'relative_error_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'relative_error_dfedu0_01_pout0_2'